In [7]:
import copy
import numpy as np
import math

class HalmaAgent:
    def __init__(self, max_depth, evaluation_function):
        self.max_depth = max_depth
        self.evaluation_function = evaluation_function
        self.best_move_max = None
        self.best_move_min = None
        
    def minimax(self, game, max_depth, player, alpha, beta):
        if max_depth == 0:
            if player == 1 and self.best_move_max:
                value = self.evaluation_function(self.best_move_max)
                return value, self.best_move_max, self.best_move_min
                
            elif player == 2 and self.best_move_min:
                value = self.evaluation_function(self.best_move_min)
                return value, self.best_move_max, self.best_move_min
        
        if player == 2:
            value = float('-inf')
            possible_moves_2 = game.moves_check(game, 2)
            for move in possible_moves_2:
                evaluation,_,_ = self.minimax(move, max_depth - 1, 1, alpha, beta)
                if evaluation > value:
                    self.best_move_max = move
                value = max(value, evaluation)
                alpha = max(alpha, value)
                if beta <= alpha:
                    break
            return value, self.best_move_max, self.best_move_min
        
        elif player == 1:
            value = float('inf')
            possible_moves_1 = game.moves_check(game, 1)
            for move in possible_moves_1:
                evaluation,_,_ = self.minimax(move, max_depth - 1, 2, alpha, beta)
                if evaluation < value:
                    self.best_move_min = move
                value = min(value, evaluation)
                beta = min(beta, value)
                if beta <= alpha:
                    break
            return value, self.best_move_max, self.best_move_min

    def choose_move(self, game):
        alpha = float('-inf')
        beta = float('inf')
        value, best_move_max, best_move_min = self.minimax(game, self.max_depth, game.current_player, alpha, beta)
        return value, best_move_max, best_move_min


In [8]:
from copy import deepcopy
from copy import copy
import numpy as np


NUM_ROWS = 6
NUM_COLUMNS = 6
#TWO_P = [5, 5, 4, 3, 2]
TWO_P = [2,2]

class HalmaGame:
    def __init__(self):
        self.goal_board = np.zeros([NUM_ROWS, NUM_COLUMNS])
        self.board = np.zeros([NUM_ROWS, NUM_COLUMNS])
        for i in range(len(TWO_P)):
            self.goal_board[i, :TWO_P[i]] = 2
            self.goal_board[-i - 1, -TWO_P[i]:] = 1
            self.board[i, :TWO_P[i]] = 1
            self.board[-i - 1, -TWO_P[i]:] = 2
        self.current_player = 1
        self.winner = -1

        self.child_boards = []
        self.parent = None
        self.parent_operator = None
        self.jump_flag = 0
        self.new_idx = None

    def __eq__(self, other):
        return np.array_equal(self.board, other.board)

    def __hash__(self):
        return hash(str([item for sublist in self.board for item in sublist]))
        #return hash(list(self.board))
    # def check_winner(self):
    #    goal_player_1 = np.where(self.goal_board == 1)
    #    goal_player_2 = np.where(self.goal_board == 2)
    #    board_index_1 = np.where(self.board == 1)
    #    board_index_2 = np.where(self.board == 2)
    #    if np.array_equal(goal_player_1, board_index_1):
    #        return self.winner = 1
    #    elif np.array_equal(goal_player_2, board_index_2):
    #        return self.winner = 2
    #    else:
    #        return self.winner = -1
    
    def set_player(self, p):
        self.player = p
        
    def set_board(self, b):
        self.board = b
        
    #def board_mod(self, idxs: list):
        
    def switch_player(self, state):
        if state.current_player == 1:
            return 2
        if state.current_player == 2:
            return 1

    def moves_check(self, state,player):
        visited = set()
        visited.add(state)
        #fck = deepcopy(state)
        #fck = state
        queue = [state]
        all_kids = set()
        jump_flag = 0
        #if state.current_player == -1:
        #    if player == 1:
        #        player = 2
        #    elif player == 2:
        #        player = 1
        #player = state.current_player
        pieces = np.array(np.where(state.board == player)).T

        for idx in pieces:
            # up
            if idx[0] - 1 >= 0:
                if not state.board[idx[0] - 1, idx[1]]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0] - 1, idx[1]] = player
                    child.parent = state
                    child.parent_operator = "up"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('up, idx: ', idx)



            # up_right
            if idx[0] - 1 >= 0 and idx[1] + 1 < NUM_COLUMNS:
                if not state.board[idx[0] - 1, idx[1] + 1]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0] - 1, idx[1] + 1] = player
                    child.parent = state
                    child.parent_operator = "up_right"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('up_right, idx: ', idx)

            # right
            if idx[1] + 1 < NUM_COLUMNS:
                if not state.board[idx[0], idx[1] + 1]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0], idx[1] + 1] = player
                    child.parent = state
                    child.parent_operator = "right"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('right, idx: ', idx)


            # down_right
            if idx[0] + 1 < NUM_ROWS and idx[1] + 1 < NUM_COLUMNS:
                if not state.board[idx[0] + 1, idx[1] + 1]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0] + 1, idx[1] + 1] = player
                    child.parent = state
                    child.parent_operator = "down_right"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('down_right, idx: ', idx)

            # down
            if idx[0] + 1 < NUM_ROWS:
                if not state.board[idx[0] + 1, idx[1]]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0] + 1, idx[1]] = player
                    child.parent = state
                    child.parent_operator = "down"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('down, idx: ', idx)

            # down left
            if idx[0] + 1 < NUM_ROWS and idx[1] - 1 >= 0:
                if not state.board[idx[0] + 1, idx[1] - 1]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0] + 1, idx[1] - 1] = player
                    child.parent = state
                    child.parent_operator = "down_left"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('down_left, idx: ', idx)

            # left
            if idx[1] - 1 >= 0:
                if not state.board[idx[0], idx[1] - 1]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    #print(child.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0], idx[1] - 1] = player
                    child.parent = state
                    child.parent_operator = "left"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('left, idx: ', idx)

            # up_left
            if idx[0] - 1 >= 0 and idx[1] - 1 >= 0:
                if not state.board[idx[0] - 1, idx[1] - 1]:
                    child = HalmaGame()
                    #child.board = fck.board.copy()
                    child.board = deepcopy(state.board)
                    child.board[idx[0], idx[1]] = 0
                    child.board[idx[0] - 1, idx[1] - 1] = player
                    child.parent = state
                    child.parent_operator = "up_left"
                    child.current_player = self.switch_player(state)
                    all_kids.add(child)
                    #print('up_left, idx: ', idx)

            while len(queue):
                node = queue.pop(0)
                #pcs = np.array(np.where(node.board == player)).T
                # jump up
                if node.jump_flag:
                    idx = node.new_idx
                if idx[0] - 2 >= 0:
                    if node.board[idx[0] - 1, idx[1]] and not node.board[idx[0] - 2, idx[1]]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0] - 2, idx[1]] = player
                        if child not in visited:
                            child.parent = node
                            child.parent_operator = "jump_up"
                            child.jump_flag = 1
                            child.new_idx = [idx[0] - 2, idx[1]]
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0


                # jump up right
                if idx[0] - 2 >= 0 and idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0] - 1, idx[1] + 1] and not node.board[idx[0] - 2, idx[1] + 2]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0] - 2, idx[1] + 2] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0] - 2, idx[1] + 2]
                            child.parent_operator = "jump_up_right"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0
                # jump right
                if idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0], idx[1] + 1] and not node.board[idx[0], idx[1] + 2]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0], idx[1] + 2] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0], idx[1] + 2]
                            child.parent_operator = "jump_right"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0

                # jump down right
                if idx[0] + 2 < NUM_ROWS and idx[1] + 2 < NUM_COLUMNS:
                    if node.board[idx[0] + 1, idx[1] + 1] and not node.board[idx[0] + 2, idx[1] + 2]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0] + 2, idx[1] + 2] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0] + 2, idx[1] + 2]
                            child.parent_operator = "jump_down_right"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0

                # jump down
                if idx[0] + 2 < NUM_ROWS:
                    if node.board[idx[0] + 1, idx[1]] and not node.board[idx[0] + 2, idx[1]]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0] + 2, idx[1]] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0] + 2, idx[1]]
                            child.parent_operator = "jump_down"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0

                # jump down left
                if idx[0] + 2 < NUM_ROWS and idx[1] - 2 >= 0:
                    if node.board[idx[0] + 1, idx[1] - 1] and not node.board[idx[0] + 2, idx[1] - 2]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board

                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0] + 2, idx[1] - 2] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0] + 2, idx[1] - 2]
                            child.parent_operator = "jump_down_left"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0
                # jump left
                if idx[1] - 2 >= 0:
                    if node.board[idx[0], idx[1] - 1] and not node.board[idx[0], idx[1] - 2]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board = deepcopy(node.board)
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0], idx[1] - 2] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0], idx[1] - 2] 
                            child.parent_operator = "jump_left"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0
                        
                # jump up left
                if idx[1] - 2 >= 0 and idx[0] -2 >= 0:
                    if node.board[idx[0]-1, idx[1] - 1] and not node.board[idx[0]-2, idx[1] - 2]:
                        #child = HalmaGame()
                        child = deepcopy(node)
                        #child.board = node.board
                        child.board[idx[0], idx[1]] = 0
                        child.board[idx[0]-2, idx[1] - 2] = player
                        if child not in visited:
                            child.parent = node
                            child.new_idx = [idx[0]-2, idx[1] - 2]
                            child.parent_operator = "jump_up_left"
                            child.jump_flag = 1
                            #child.current_player = self.switch_player(state)
                            visited.add(child)
                            queue.append(child)
                            all_kids.add(child)
                    else:
                        node.current_player = self.switch_player(state)
                        node.jump_flag = 0
            queue.append(state)
        state.child_boards = all_kids
        return all_kids

In [9]:
# Define evaluation functions

def evaluation_function_1(game):
    board_goal = game.goal_board
    board_state = game.board
    indexes_target_2 = np.where(board_goal == 2)
    indexes_in_2 = np.where(board_goal == 1)
    indexes_state_2 = np.where(board_state == 2)
    indexes_state_1 = np.where(board_state == 1)
    index_list_inital_1 = []
    index_list_target_1 = []
    index_list_inital_2 = []
    index_list_target_2 = []
    index_list_2 = []
    index_list_1 = []
    
    for i in range(len(indexes_in_2[0])):
        index_in_2 = [indexes_in_2[0][i], indexes_in_2[1][i]]
        index_list_inital_2.append(index_in_2)
        index_tg_2 = [indexes_target_2[0][i], indexes_target_2[1][i]]
        index_list_target_2.append(index_tg_2)
        index_2 = [indexes_state_2[0][i], indexes_state_2[1][i]]
        index_list_2.append(index_2)
        index_1 = [indexes_state_1[0][i], indexes_state_1[1][i]]
        index_list_1.append(index_1)

    index_list_inital_1 = index_list_target_2
    index_list_target_1 = index_list_inital_2

    min_dist_1 = []
    min_dist_2 = []
    for i in range(len(index_list_inital_2)):
        dist_it_1 = []
        dist_it_2 = []
        for j in range(len(index_list_target_2)):
            if index_list_1[i] not in index_list_target_1:
                dist_it_1.append(math.dist(index_list_1[i], index_list_target_1[j]))
                dist_it_2.append(math.dist(index_list_2[i], index_list_target_2[j]))
        min_dist_1.append(max(dist_it_1))
        min_dist_2.append(max(dist_it_2))

    distances_1 = round(sum(min_dist_1), 5)
    distances_2 = round(sum(min_dist_2), 5)
    
    diff = distances_1 - distances_2

    return diff

# Create two agents with different levels of difficulty

game = HalmaGame()
#x = game.moves_check(game, game.current_player)

agent1 = HalmaAgent(max_depth=2, evaluation_function=evaluation_function_1)
agent2 = HalmaAgent(max_depth=4, evaluation_function=evaluation_function_1)
#print(evaluation_function_1(x[0]))

# Run the game until it's over
value1, best_move_max1, best_move_min1 = agent1.choose_move(game)
print('value1', value1)
print('best move min1', best_move_min1.board)#
print('best move max1', best_move_max1.board)


AttributeError: 'HalmaGame' object has no attribute 'board'